In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Convert JSON to DataFrame
df = pd.read_json("chat.json")

df["conversations"]

0         [{'from': 'human', 'value': 'Provide a brief d...
1         [{'from': 'human', 'value': '<image>
Write a t...
2         [{'from': 'human', 'value': 'Share a concise i...
3         [{'from': 'human', 'value': 'Relay a brief, cl...
4         [{'from': 'human', 'value': '<image>
Render a ...
                                ...                        
595370    [{'from': 'human', 'value': 'Present a compact...
595371    [{'from': 'human', 'value': '<image>
Summarize...
595372    [{'from': 'human', 'value': 'Describe the imag...
595373    [{'from': 'human', 'value': '<image>
Create a ...
595374    [{'from': 'human', 'value': '<image>
Provide a...
Name: conversations, Length: 595375, dtype: object

In [ ]:
import pandas as pd
import plotly.express as px

# Calculate the length of each text
df['text_len'] = df['conversations'].apply(lambda x: len(x[0]["value"]) + len(x[1]["value"]))

# Histogram
fig_hist = px.histogram(df, x='text_len', nbins=40, title='Distribution of Text Lengths - Histogram')
fig_hist.show()

# Box Plot
fig_box = px.box(df, y='text_len', title='Distribution of Text Lengths - Box Plot')
fig_box.show()


In [ ]:
import pandas as pd
import plotly.express as px
from transformers import MBart50TokenizerFast
tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50-many-to-many-mmt"
)
# Calculate the length of each text
df['mbart-large-50-many-to-many-mmt_tokens_len'] = df['conversations'].apply(lambda x: len(tokenizer(x[0]["value"] + tokenizer.pad_token + x[1]["value"])["input_ids"]))

# Histogram
fig_hist = px.histogram(df, x='mbart-large-50-many-to-many-mmt_tokens_len', nbins=40, title='Distribution of mbart-large-50-many-to-many-mmt tokens count - Histogram')
fig_hist.show()

# Box Plot
fig_box = px.box(df, y='mbart-large-50-many-to-many-mmt_tokens_len', title='Distribution of mbart-large-50-many-to-many-mmt tokens count - Box Plot')
fig_box.show()
## MAX

In [ ]:
import pandas as pd
import plotly.express as px
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/madlad400-3b-mt")
# Calculate the length of each text
df['madlad400_tokens_len'] = df['conversations'].apply(lambda x: len(tokenizer(x[0]["value"] + tokenizer.pad_token + x[1]["value"])["input_ids"]))

# Histogram
fig_hist = px.histogram(df, x='madlad400_tokens_len', nbins=40, title='Distribution of madlad400-3b-mt tokens count - Histogram')
fig_hist.show()

# Box Plot
fig_box = px.box(df, y='madlad400_tokens_len', title='Distribution of madlad400-3b-mt tokens count - Box Plot')
fig_box.show()


In [ ]:
df = df[(df['mbart-large-50-many-to-many-mmt_tokens_len'] <= 64) &  (df['mbart-large-50-many-to-many-mmt_tokens_len']<=64)]

In [ ]:
df["conversations"][0]

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = 'jbochi/madlad400-3b-mt'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

text = "<2pl> Provide a brief description of the given image.\n<image>."
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(input_ids=input_ids)

tokenizer.decode(outputs[0], skip_special_tokens=True)
# Eu adoro pizza!


'dpowiedź: Podaj krótki opis danego obrazu<<<<<'

In [ ]:
df.shape[0]%32

In [7]:
def process_batches(df: pd.DataFrame, batch_size=32) -> list:
    """Batches list of data for training"""
    batches_list = []
    for i in range(df.shape[0]//batch_size):
        ldx = i*batch_size
        batches_list.append(df["conversations"][ldx:ldx+batch_size])

    batches_list.append(df[-(df.shape[0]%batch_size):]) 
    return batches_list

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast



article_hi = "Provide a brief description of the given image.\n<image>."

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Hindi to French
tokenizer.src_lang = "en_EN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["pl_PL"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


In [ ]:
b.apply(lambda x: x[0]["value"]).to_list()

In [8]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = 'jbochi/madlad400-3b-mt'
model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="cuda:0")
tokenizer = T5Tokenizer.from_pretrained(model_name, device_map="cuda:0")

text = "<2pl> Provide a brief description of the given image. \n"
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(input_ids=input_ids)

tokenizer.decode(outputs[0], skip_special_tokens=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 10.00 GiB total capacity; 22.07 GiB already allocated; 0 bytes free; 23.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
batches = process_batches(df=df, batch_size=32)
trans_token = "<2pl> "
for b in batches:
    encoded_human = tokenizer(b.apply(lambda x: trans_token + x[0]["value"].replace("<image>",'').replace("\n","")).to_list(), return_tensors="pt", max_length=64, padding=True)
    encoded_ai = tokenizer(b.apply(lambda x: trans_token + x[1]["value"]).replace("<image>",'').replace("\n","").to_list(), return_tensors="pt", max_length=64, padding=True)
    break

text =  encoded_human.to("cuda")
print(text)
outputs = model.generate(**text)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

{'input_ids': tensor([[   805,    113, 190947,  ...,      1,      1,      1],
        [   805,    113,   1542,  ...,    807,      2,      1],
        [   805,    113,  34666,  ...,      1,      1,      1],
        ...,
        [   805,    113, 156303,  ...,      2,      1,      1],
        [   805,    113,   1544,  ...,    807,      2,      1],
        [   805,    113,  34666,  ...,      1,      1,      1]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}


['Podaj krótki opis danego obrazu.',
 'Napisz zwięzłe, ale pouczające podsumowanie obrazu.',
 'Podziel się zwięzłą interpretacją dostarczonych obrazów.',
 'Przekaż krótki, jasny opis pokazanego obrazu.',
 'Wykonaj jasne i zwięzłe podsumowanie zdjęcia.',
 'Stwórz zwartą narrację reprezentującą prezentowany obraz.',
 'Wykonaj jasne i zwięzłe podsumowanie zdjęcia.',
 'Podaj krótkie i jasne wyjaśnienie następującego obrazu.',
 'Stwórz zwartą narrację reprezentującą prezentowany obraz.',
 'Podsumuj wizualną zawartość obrazu.',
 'Podaj krótki opis danego obrazu.',
 'Podziel się zwięzłą interpretacją dostarczonych obrazów.',
 'Stwórz zwartą narrację reprezentującą prezentowany obraz.',
 'Podaj krótki opis danego obrazu.',
 'Przekaż krótki, jasny opis pokazanego obrazu.',
 'Stwórz zwartą narrację reprezentującą prezentowany obraz.',
 'Podsumuj wizualną zawartość obrazu.',
 'Podziel się zwięzłą interpretacją dostarczonych obrazów.',
 'Podsumuj wizualną zawartość obrazu.',
 'Opisz obraz zwięźle.

In [24]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [12]:
translation

'Podaj krótki opis danego obrazu.'

In [102]:
translation

'dpowiedz na pytanie: 1.'

In [44]:
encoded_human

{'input_ids': tensor([[   805,    113, 190947,    811, 127316, 153596,   1014,   1013,  74452,
          28019,    807,      3,      2,      1,      1,      1,      1,      1,
              1,      1],
        [   805,    113,    805,      3,   2249,  17232,    811,  81080,   2590,
         149583,    816, 160783,    873,   1014,   1013, 119364,    807,      2,
              1,      1],
        [   805,    113,  34666,    811,  82635,   3295,  38995,   4115,   1014,
           1013,  28019, 169222,    807,      3,      2,      1,      1,      1,
              1,      1],
        [   805,    113,  53431,    873,    811, 127316,    806,  70792,  18640,
           1014,   1013, 119364,  15637,    824,    807,      3,      2,      1,
              1,      1],
        [   805,    113,    805,      3,   5118,   7044,    811,  70792,   1055,
          82635,   3295, 160783,    873,   1014,   1013,  20519,    807,      2,
              1,      1],
        [   805,    113, 156303,    811, 13538

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = 'jbochi/madlad400-3b-mt'
model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
tokenizer = T5Tokenizer.from_pretrained(model_name)

text = "<2pl> Create a compact narrative representing the image presented!"
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(input_ids=input_ids)

tokenizer.decode(outputs[0], skip_special_tokens=True)
# Eu adoro pizza!


In [ ]:
import torch

print(torch.cuda.is_available())

print(torch.cuda.device_count())